In [5]:
pip install transformers

     |████████████████████████████████| 1.3MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 15.1MB/s 
     |████████████████████████████████| 2.9MB 31.2MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1e5374e38685ffafc64505c5a26b1c069b3e4da83d4b36260b72c48042a39847
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
import torch

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [8]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [9]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

#Digital Advisor for the parents with Autistic Children
source: theautismblog.com

In [15]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

context1 = """Caring for children with autism spectrum disorders (ASDs) is challenging and affects family life. ASDs are neurological complex conditions impairing social interaction and communication, such as difficulties to respond to social interactions or deficits in understanding nonverbal communication. Moreover, persons diagnosed with an ASD have restricted behavioral functions, such as stereotype behavior or inability to adjust to new situations (Strock 2007). Children with ASDs often exhibit more than one of these core ASD symptoms and many also suffer from associated symptoms, such as severe tantrums or sleep problems (Aman 2004; Couturier et al. 2005). Moreover, health in general of these children is lower than that of typically developing children (Gurney et al. 2006; Kuhlt-hau et al. 2010b, 2013).
Tips: Take care of yourself. Believe. A child with autism is first and foremost a child. He or she is a growing person with unknown possibilities. Believe in what the child can do. Don’t define the child by a diagnosis. Show your love and interest. Children with ASD may have trouble showing their feelings. But they still need to know that you love them.Be affectionate and respectful. Children with ASD often need a hug, just like other children. Sometimes they need this much more than other children. But some children don't like to be touched. Respect their personal space."""

print(wrapper.fill(context1))

Caring for children with autism spectrum disorders (ASDs) is challenging and
affects family life. ASDs are neurological complex conditions impairing social
interaction and communication, such as difficulties to respond to social
interactions or deficits in understanding nonverbal communication. Moreover,
persons diagnosed with an ASD have restricted behavioral functions, such as
stereotype behavior or inability to adjust to new situations (Strock 2007).
Children with ASDs often exhibit more than one of these core ASD symptoms and
many also suffer from associated symptoms, such as severe tantrums or sleep
problems (Aman 2004; Couturier et al. 2005). Moreover, health in general of
these children is lower than that of typically developing children (Gurney et
al. 2006; Kuhlt-hau et al. 2010b, 2013). Tips: Take care of yourself. Believe. A
child with autism is first and foremost a child. He or she is a growing person
with unknown possibilities. Believe in what the child can do. Don’t define

In [11]:
question = "what is asd"

answer_question(question, context1)

Query has 173 tokens.

Answer: "neurological complex conditions impairing social interaction and communication"


In [16]:
question = "what are some tips for treating a kid"

answer_question(question, context1)

Query has 300 tokens.

Answer: "take care of yourself . believe . a child with autism is first and foremost a child . he or she is a growing person with unknown possibilities . believe in what the child can do . don ’ t define the child by a diagnosis . show your love and interest"


In [18]:
question = "what are the troubles faced by an autistic kid?"

answer_question(question, context1)

Query has 304 tokens.

Answer: "severe tantrums or sleep problems"
